In [2]:
'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, prob_dropout=0.3):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        # mod here
        self.dropout1 = nn.Dropout2d(p=prob_dropout)
        
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        

        # mod here
        self.dropout2 = nn.Dropout2d(p=prob_dropout)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet9():
    return ResNet(BasicBlock, [1, 1, 1, 1])
    
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

# test()



print("done")

done


In [3]:
from torchvision import transforms, datasets
import torch.utils.data
from torchvision.transforms import ToTensor
torch.manual_seed(1024)
# Define transformations
train_transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1), # Add color jitter
    transforms.RandomHorizontalFlip(), #apply horizontal flipping
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(), # Randomly flip the images on the horizontal axis
    transforms.RandomRotation(10), # Randomly rotate the images by +/- 10 degrees
    transforms.RandomCrop(32, padding=4), # Apply random crops
    transforms.ToTensor(), # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # Normalize images
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Apply transformations to datasets
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

batch_size = 16
# Create DataLoader
trainDataLoader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testDataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

print("done")

Files already downloaded and verified
Files already downloaded and verified
done


In [4]:
# pip install torchsummary

In [5]:
import torch.optim.lr_scheduler as lr_scheduler
from torchsummary import summary
import pickle
import pandas as pd
import numpy as np
learn_rate = 0.001
num_epochs = 30
model = ResNet9().cuda()
model.load_state_dict(torch.load('bestest_learn_rate_change_model.pth'))
loss = torch.nn.CrossEntropyLoss() # Step 2: loss
optimizer = torch.optim.SGD(model.parameters(), lr=learn_rate, momentum=0.9, weight_decay=1e-5)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)  # T_max is typically set to the number of epochs
print("done")

done


In [6]:
# KAGGLE CHECKER
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pickle
from PIL import Image
import matplotlib.pyplot as plt
torch.manual_seed(1024)

def calculate_accuracy(predictions):
    actual = []
    pattern = [8, 2, 9, 0, 4, 3, 6, 1, 7, 5]
    for num in pattern:
        actual.extend([num] * 1000)
    num_matches = sum(1 for pred, act in zip(predictions, actual) if pred == act)
    accuracy = (num_matches / len(predictions)) * 100
    return accuracy

def load_kaggle_data(file, transform=None):
    import pickle
    with open(file, 'rb') as fo:
        batch = pickle.load(fo, encoding='bytes')
    images = batch[b'data']
    images = images.reshape((-1, 3, 32, 32)).transpose(0, 2, 3, 1)
    min_val = np.min(images)
    max_val = np.max(images)
    images = (images - min_val) / (max_val - min_val) #normalizing facepalm
    processed_images=[]
    for i,image in enumerate(images):
        pil_image = Image.fromarray((image * 255).astype(np.uint8))
        # Apply transformation
        pil_image = transform(pil_image)
        
        # Append transformed image
        processed_images.append(pil_image)
    return processed_images
kaggle_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_images = load_kaggle_data('kaggle/cifar_test_nolabels.pkl', transform=kaggle_transform)
kaggleLoader = torch.utils.data.DataLoader(test_images, batch_size=1,shuffle=False)
print("done")

done


In [ ]:
predictions = []
# model.eval()
for image in kaggleLoader:
    image = image.cuda()
    with torch.no_grad():
        output = model(image)
        predictions.append(output.argmax().item())

print(len(predictions))
# print(calculate_accuracy(predictions))
best_existing_accuracy = calculate_accuracy(predictions)
best_kagg_acc = best_existing_accuracy

In [ ]:
train_loss_history = []
test_loss_history = []
matches = 0
total = 0
num_epochs=50

print("begginning training arc now!~~~ \n")
for epoch in range(num_epochs):
    train_loss = 0.0
    test_loss = 0.0
    
    model.train()
    for i, data in enumerate(trainDataLoader):
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        optimizer.zero_grad() # zero out any gradient values from the previous iteration
        predicted_output = model(images) # forward propagation
        fit = loss(predicted_output, labels)  # calculate our measure of goodness
        fit.backward() # backpropagation
        optimizer.step() # update the weights of our trainable parameters
        train_loss += fit.item()
    train_loss = train_loss / len(trainDataLoader)
    train_loss_history += [train_loss]
    print("epoch " + str(epoch) + " train loss " + str(train_loss))
    scheduler.step()

    predictions = []
    model.eval()
    for image in kaggleLoader:
        image = image.cuda()
        with torch.no_grad():
            output = model(image)
        predictions.append(output.argmax().item())
    curr_acc = calculate_accuracy(predictions)
    if curr_acc > best_kagg_acc:
        lr_best = learn_rate
        best_kagg_acc = curr_acc
        torch.save(model.state_dict(), 'bestest_learn_rate_change_model.pth')
    print("finished round " + str(epoch) + " with accuracy of " + str(curr_acc))
print("done")

begginning training arc now!~~~ 

epoch 0 train loss 0.07065265097083058
finished round 0 with accuracy of 80.96
epoch 1 train loss 0.06826192733701318
finished round 1 with accuracy of 80.65


In [8]:
model.eval()
with torch.no_grad():
    correct, total = 0, 0
    for images, labels in testDataLoader:
        images = images.cuda()
        labels = labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Test accuracy: {100 * correct / total}%')

Test accuracy: 92.51%


In [15]:
# bestest_learn_rate_change_model
# model = torch.load('bestest_learn_rate_change_model.pth')
model.load_state_dict(torch.load('bestest_learn_rate_change_model.pth'))

<All keys matched successfully>

In [16]:
predictions = []
model.eval()
for image in kaggleLoader:
    image = image.cuda()
    with torch.no_grad():
        output = model(image)
        predictions.append(output.argmax().item())

print(len(predictions))

10000


In [17]:
print(calculate_accuracy(predictions))

80.93


In [18]:
# Create a DataFrame with IDs and labels
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
test_images_nl_id = unpickle('kaggle/cifar_test_nolabels.pkl')[b'ids'].tolist()
df = pd.DataFrame({
    'ID': test_images_nl_id,
    'Labels': predictions
})

# Save the DataFrame to a CSV file
df.to_csv('predictions.csv', index=False)

print("done")

done


In [14]:
# can be used to store training loss and accuracy to later plot

# Save the list to a variable
with open('train_loss_history.py', 'w') as f:
    f.write(str(train_loss_history))

# Now you can access the list in your Jupyter notebook:
import train_loss_history
print(train_loss_history.train_loss_history)  


AttributeError: module 'train_loss_history' has no attribute 'train_loss_history'

In [ ]:
# Import necessary libraries
import nbformat
from nbconvert import HTMLExporter

# Get the current notebook object
nb = nbformat.read(filename='cleannotebook.ipynb', as_version=4)

# Export the notebook to HTML
html_exporter = HTMLExporter()
(body, resources) = html_exporter.from_notebook_node(nb)

# Save the HTML output
with open('output.html', 'w') as f:
   f.write(body)